In [89]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
import collections
import nest_asyncio
%reload_ext tensorboard
nest_asyncio.apply()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

In [2]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data(only_digits=True)

In [83]:
 def setServerTestDataTFF(serverTestData):  #Creates a test set for server
        BATCH_SIZE = 500
        SHUFFLE_BUFFER = 100
        PREFETCH_BUFFER= 10

        def preprocess(dataset):
            def batch_format_fn(element):
                """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
                return collections.OrderedDict(
                    x=tf.reshape(element['pixels'], [-1, 784]),
                    y=tf.reshape(element['label'], [-1, 1]))

            return dataset.repeat(10).shuffle(SHUFFLE_BUFFER).batch(
              BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)
        
        serverData1 = serverTestData.create_tf_dataset_for_client(serverTestData.client_ids[10])
        for i in range (11,60):
            serverData1 = serverData1.concatenate(serverTestData.create_tf_dataset_for_client(serverTestData.client_ids[i]))
            
        serverData =tfds.as_numpy(preprocess(serverData1))
        return serverData

In [84]:
serverDS= setServerTestDataTFF(emnist_test)

In [85]:
serverDS

In [65]:
serverDS.next()

AttributeError: '_IterableDataset' object has no attribute 'next'

In [86]:
testData = next(iter(serverDS))

In [87]:
len(testData['x'])

500

In [88]:
testData

OrderedDict([('x',
              array([[1., 1., 1., ..., 1., 1., 1.],
                     [1., 1., 1., ..., 1., 1., 1.],
                     [1., 1., 1., ..., 1., 1., 1.],
                     ...,
                     [1., 1., 1., ..., 1., 1., 1.],
                     [1., 1., 1., ..., 1., 1., 1.],
                     [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)),
             ('y',
              array([[6],
                     [9],
                     [4],
                     [1],
                     [8],
                     [3],
                     [9],
                     [6],
                     [4],
                     [5],
                     [1],
                     [9],
                     [2],
                     [8],
                     [3],
                     [3],
                     [2],
                     [3],
                     [9],
                     [7],
                     [3],
                     [9],
                     [7],
        

In [30]:
m=emnist_test.create_tf_dataset_for_client(emnist_test.client_ids[30])

In [44]:
print( len(m))
print(m.cardinality().numpy())

13
13


In [39]:
k=emnist_test.create_tf_dataset_for_client(emnist_test.client_ids[20])

In [40]:
len(k)

11

In [50]:
m=m.concatenate(k)

In [52]:
print( len(m))
print(m.cardinality().numpy())

24
24


In [57]:
serverData = emnist_test.create_tf_dataset_for_client(emnist_test.client_ids[10])
for i in range (11,60):
    serverData = serverData.concatenate(emnist_test.create_tf_dataset_for_client(emnist_test.client_ids[i]))

In [58]:
print( len(serverData))
print(serverData.cardinality().numpy())

568
568
